In [1]:
import sys
sys.path.append(r"C:\Users\Vivian\Documents\CONCH\conch")

from open_clip_custom import create_model_from_pretrained, tokenize, get_tokenizer
import torch
import os
from PIL import Image
from pathlib import Path
from tqdm import tqdm

# show all jupyter output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

c:\Users\Vivian\anaconda3\envs\conch\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Finetuning

In [2]:
# from conch.open_clip_custom import create_model_from_pretrained, tokenize, get_tokenizer
from torch import nn
import os
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import pandas as pd
# import glob

import matplotlib.pyplot as plt
import numpy as np

# import tqdm

# import skimage

from torch.utils.data import DataLoader, Dataset

import os
from torchvision import transforms
# import timm
from huggingface_hub import login, hf_hub_download

# show all jupyter output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

load conch

In [3]:
class CONCHModelForFinetuning(nn.Module):
    def __init__(self, num_classes=8, config={'hidden_size': 512}): # change number of classes for each dataset(8 for breast, 2 for breast)
        super().__init__()
        self.config = config
        self.model = self.make_conch()
        # self.fc = nn.Linear(self.config['hidden_size'], num_classes) # full finetuning?

        # linear probing
        # Freeze all parameters in the backbone
        # for param in self.model.parameters():
        #     param.requires_grad = False

        # Only this linear layer will be trained
        self.fc = nn.Linear(self.config['hidden_size'], num_classes)

    def make_conch(self):
        # Load the model from "create_model_from_pretrained"
        model_cfg = 'conch_ViT-B-16'
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # checkpoint_path = 'checkpoints/CONCH/pytorch_model.bin'
        checkpoint_path = 'C:\\Users\\Vivian\\Documents\\CONCH\\checkpoints\\conch\\pytorch_model.bin' # load in checkpoint here
        # checkpoint_path = r'C:\Users\Vivian\Documents\CONCH\_finetune_weights\Fold2_F_PT_model.pth' # loading breakhis finetuned model
        model, preprocess = create_model_from_pretrained(model_cfg, checkpoint_path, device=device)
        
        return model
        
    def forward(self, x):
        out, h = self.model.visual(x)
        return self.fc(out)

In [4]:
model = CONCHModelForFinetuning().to('cuda')

C:\Users\Vivian\Documents\CONCH\conch\open_clip_custom\factory.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=map

Load UNi

In [3]:
import os
import torch
from torchvision import transforms
import timm

class UNIModelForFinetuning(nn.Module):
    def __init__(self, num_classes=8,checkpoint_path=None): # change number of classes accordingly 
        ## ************ change for UNI ************
        super().__init__()
        # self.config = config
        self.model = self.make_uni()
        # self.fc = nn.Linear(self.config['hidden_size'], num_classes) # keep commented
        # self.fc = nn.Linear(1024, num_classes)  # Match Vision Transformer output # full finetuning

        #***** Freeze all backbone parameters - linear probing *****
        # for param in self.model.parameters():
        #     param.requires_grad = False

        # Add a trainable classification head
        self.fc = nn.Linear(1024, num_classes)

        # ----- Load checkpoint if needed -----
        if checkpoint_path:
            print(f"Loading checkpoint from: {checkpoint_path}")
            self.load_state_dict(torch.load(checkpoint_path, map_location='cuda'))
    
    def make_uni(self):
        local_dir = r"C:\Users\Vivian\Documents\CONCH\checkpoints\uni" # load in UNI model
        os.makedirs(local_dir, exist_ok=True)  # create directory if it does not exist
        
        model = timm.create_model(
            "vit_large_patch16_224", img_size=224, patch_size=16, init_values=1e-5, num_classes=0, dynamic_img_size=True
        )
        model.load_state_dict(torch.load(os.path.join(local_dir, "pytorch_model.bin"), map_location="cpu"), strict=True)
        
        return model 
    
    def forward(self, x):
        out = self.model(x)
        return self.fc(out)

In [4]:
model = UNIModelForFinetuning().to('cuda')

C:\Users\Vivian\AppData\Local\Temp\ipykernel_29020\969011001.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(local_dir, "p

In [5]:
from torch.utils.data import Dataset
# import skimage.io
import skimage.transform
import matplotlib.pyplot as plt

class HistopathologyDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe.reset_index(drop=True)
        self.transform = transform
        self.label_map = {
            'B_A': 0,
            'B_F': 1,
            'B_PT': 2,
            'B_TA': 3,
            'M_DC': 4,
            'M_LC': 5,
            'M_MC': 6,
            'M_PC': 7
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = row['image']
        class_name = row['class']
        subclass_name = row['subclass']
        label = self.label_map[class_name + '_' + subclass_name]

        # image = skimage.io.imread(img_path)
        # image = skimage.transform.resize(image, (224, 224), anti_aliasing=True)
        # image = image.transpose((2, 0, 1))  # HWC → CHW
        # image = image.astype(np.float32) # Ensure float32

        # image = plt.imread(img_path)
        # image = skimage.transform.resize(image, (224, 224))
        # image = image.transpose((2, 0, 1))
        # image = image.astype(np.float32)

        try:
        #     image = Image.open(img_path).convert("RGB")
        #     image = image.resize((224, 224))
        #     image = np.array(image).transpose((2, 0, 1))  # CHW format
        #     image = torch.tensor(image, dtype=torch.float32) / 255.0

            image = plt.imread(img_path)
            image = skimage.transform.resize(image, (224, 224))
            image = image.transpose((2, 0, 1))
            image = image.astype(np.float32)
        except Exception as e:
            print(f"⚠️ Error loading image at {img_path}: {e}")
            return self.__getitem__((idx + 1) % len(self))  # Try next image


        if self.transform:
            image = self.transform(image)

        return image, label


run this

In [6]:
import os
import torch
import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report


def run_k_fold_finetuning(
    model,
    k_folds,
    train_loader_fn,
    test_loader_fn,
    model_save_dir,
    results_csv_path,
    device='cuda',
    lr=2e-5,
    num_epochs=5,
    patience=5,
    is_binary=True,
    save_summary=True
):
    os.makedirs(model_save_dir, exist_ok=True)
    os.makedirs(os.path.dirname(results_csv_path), exist_ok=True)

    all_fold_results = []
    all_per_class_f1 = {}

    for fold in range(1, k_folds + 1):
        print(f"\n=== Fold {fold}/{k_folds} ===")

        train_loader = train_loader_fn(fold)
        test_loader = test_loader_fn(fold)

        model.fc = torch.nn.Linear(512, 2 if is_binary else 8).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # Full fine-tuning
        # optimizer = torch.optim.Adam(model.fc.parameters(), lr=lr)  # Linear probing
        criterion = torch.nn.CrossEntropyLoss()
        scaler = torch.cuda.amp.GradScaler()

        best_acc = 0
        epochs_no_improve = 0
        model_path = os.path.join(model_save_dir, f"fold{fold}.pth")

        for epoch in range(num_epochs):
            model.train()
            total_loss, correct, total = 0, 0, 0
            y_true, y_pred = [], []

            for images, labels, *_ in tqdm.tqdm(train_loader, desc=f"Fold {fold} - Epoch {epoch+1}", total=len(train_loader)):
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.cuda.amp.autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                total_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())

            acc = correct / total
            print(f"Epoch {epoch+1}: Train Loss = {total_loss:.4f}, Train Acc = {acc:.4f}")

            if acc > best_acc:
                best_acc = acc
                epochs_no_improve = 0
                torch.save(model.state_dict(), model_path)
                print(f"✅ Model saved at epoch {epoch+1} with train acc: {acc:.4f}")
            else:
                epochs_no_improve += 1
                print(f"No improvement for {epochs_no_improve} epoch(s)")
                if epochs_no_improve >= patience:
                    print("⏹️ Early stopping triggered.")
                    break

        # --- Final Test ---
        model.load_state_dict(torch.load(model_path))
        model.eval()
        y_test, y_pred_test = [], []
        with torch.no_grad():
            for images, labels, *_ in tqdm.tqdm(test_loader, desc=f"Fold {fold} [Test]"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                y_test.extend(labels.cpu().numpy())
                y_pred_test.extend(predicted.cpu().numpy())

        test_acc = (np.array(y_test) == np.array(y_pred_test)).mean()
        prec = precision_score(y_test, y_pred_test, average='binary' if is_binary else 'weighted')
        rec = recall_score(y_test, y_pred_test, average='binary' if is_binary else 'weighted')
        f1 = f1_score(y_test, y_pred_test, average='binary' if is_binary else 'weighted')

        class_report = classification_report(y_test, y_pred_test, output_dict=True)
        for cls, metrics in class_report.items():
            if cls.isdigit():  # skip 'accuracy', 'macro avg', etc.
                cls = int(cls)
                if cls not in all_per_class_f1:
                    all_per_class_f1[cls] = []
                all_per_class_f1[cls].append(metrics['f1-score'])

        result = {
            'Fold': fold,
            'Best Train Accuracy': best_acc,
            'Test Accuracy': test_acc,
            'Test Precision': prec,
            'Test Recall': rec,
            'Test F1': f1
        }
        all_fold_results.append(result)

    # Save all fold results
    results_df = pd.DataFrame(all_fold_results)
    results_df.to_csv(results_csv_path, index=False)
    print("✅ All folds complete. Results saved.")

    # --- Compute means and stds across folds ---
    avg_metrics = results_df.iloc[:, 1:].mean()
    std_metrics = results_df.iloc[:, 1:].std()

    print("\n📊 Cross-Validation Averages ± Std:")
    for col in avg_metrics.index:
        print(f"{col}: {avg_metrics[col]:.4f} ± {std_metrics[col]:.4f}")

    # --- Per-class F1 average and std ---
    class_f1_rows = []
    print("\n📊 Per-Class F1 Scores:")
    for cls, scores in all_per_class_f1.items():
        mean_f1 = np.mean(scores)
        std_f1 = np.std(scores)
        print(f"Class {cls}: {mean_f1:.4f} ± {std_f1:.4f}")
        class_f1_rows.append({"Fold": f"Class {cls}", "F1 Mean": mean_f1, "F1 Std": std_f1})

    # --- Append summary rows to CSV ---
    if save_summary:
        summary_df = pd.DataFrame([
            {'Fold': 'Average', **avg_metrics.to_dict()},
            {'Fold': 'Std Dev', **std_metrics.to_dict()}
        ])
        pd.concat([results_df, summary_df, pd.DataFrame(class_f1_rows)], axis=0).to_csv(results_csv_path, index=False)
        print(f"📁 Appended summary rows to: {results_csv_path}")


In [7]:
import pandas as pd
from torch.utils.data import DataLoader

metadata = pd.read_csv(r'C:\Users\Vivian\Documents\CONCH\breakhis_scripts\metadata_folds.csv')

def make_train_loader(fold, batch_size=8):
    train_df = metadata[metadata['fold'] != fold]
    train_dataset = HistopathologyDataset(train_df)
    return DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

def make_test_loader(fold, batch_size=8):
    test_df = metadata[metadata['fold'] == fold]
    test_dataset = HistopathologyDataset(test_df)
    return DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [8]:
from functools import partial
# metadata_path = r"C:\Users\Vivian\Documents\CONCH\breakhis_scripts\metadata_folds.csv"
model_save_dir = r"C:\Users\Vivian\Documents\CONCH\_finetune_weights_CONCH\breakhis_multiclass2"
results_csv_dir = r"C:\Users\Vivian\Documents\CONCH\breakhis_scripts\CONCH_multiclass"
results_csv_path = os.path.join(results_csv_dir, "CONCH_fullft_results2.csv")

run_k_fold_finetuning(
    model=model,  # your fine-tuning model
    k_folds=5,
    train_loader_fn=make_train_loader,
    test_loader_fn=make_test_loader,
    model_save_dir=model_save_dir,
    results_csv_path=results_csv_path,
    device='cuda',
    lr=2e-5,
    num_epochs=5,
    patience=5,
    is_binary=False
)


C:\Users\Vivian\AppData\Local\Temp\ipykernel_27992\3022664669.py:39: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()



=== Fold 1/5 ===


Fold 1 - Epoch 1:   0%|          | 0/771 [00:00<?, ?it/s]C:\Users\Vivian\AppData\Local\Temp\ipykernel_27992\3022664669.py:54: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Fold 1 - Epoch 1: 100%|██████████| 771/771 [02:43<00:00,  4.72it/s]


Epoch 1: Train Loss = 425.3006, Train Acc = 0.8122
✅ Model saved at epoch 1 with train acc: 0.8122


Fold 1 - Epoch 2: 100%|██████████| 771/771 [02:44<00:00,  4.67it/s]


...

Fold 5 - Epoch 5: 100%|██████████| 792/792 [02:48<00:00,  4.70it/s]


Epoch 5: Train Loss = 23.7792, Train Acc = 0.9815
✅ Model saved at epoch 5 with train acc: 0.9815


C:\Users\Vivian\AppData\Local\Temp\ipykernel_27992\3022664669.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
Fold 5 [Test

✅ All folds complete. Results saved.

📊 Cross-Validation Averages ± Std:
Best Train Accuracy: 0.9816 ± 0.0062
Test Accuracy: 0.8315 ± 0.1607
Test Precision: 0.8668 ± 0.1370
Test Recall: 0.8315 ± 0.1607
Test F1: 0.8341 ± 0.1580

📊 Per-Class F1 Scores:
Class 0: 0.6431 ± 0.3883
Class 1: 0.8050 ± 0.1639
Class 2: 0.4564 ± 0.4586
Class 3: 0.8414 ± 0.1371
Class 4: 0.8801 ± 0.1051
Class 5: 0.5644 ± 0.4418
Class 6: 0.5326 ± 0.3584
Class 7: 0.5547 ± 0.3710
📁 Appended summary rows to: C:\Users\Vivian\Documents\CONCH\breakhis_scripts\CONCH_multiclass\CONCH_fullft_results2.csv



c:\Users\Vivian\anaconda3\envs\conch\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Vivian\anaconda3\envs\conch\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Vivian\anaconda3\envs\conch\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Vivian\anaconda3\e

In [8]:
from functools import partial
# metadata_path = r"C:\Users\Vivian\Documents\CONCH\breakhis_scripts\metadata_folds.csv"
# model_save_dir = r"C:\Users\Vivian\Documents\CONCH\_finetune_weights_CONCH\breakhis_multiclass"
# results_csv_dir = r"C:\Users\Vivian\Documents\CONCH\breakhis_scripts\CONCH_multiclass"
# results_csv_path = os.path.join(results_csv_dir, "CONCH_fullft_results.csv")
metadata_path = r"C:\Users\Vivian\Documents\CONCH\breakhis_scripts\metadata_folds.csv"
model_save_dir = r"C:\Users\Vivian\Documents\CONCH\_finetune_weights_UNI\breakhis_multiclass"
results_csv_dir = r"C:\Users\Vivian\Documents\CONCH\breakhis_scripts\UNI_multiclass"
results_csv_path = os.path.join(results_csv_dir, "UNI_fullft_results.csv")

run_k_fold_finetuning(
    model=model,  # your fine-tuning model
    k_folds=5,
    train_loader_fn=make_train_loader,
    test_loader_fn=make_test_loader,
    model_save_dir=model_save_dir,
    results_csv_path=results_csv_path,
    device='cuda',
    lr=2e-5,
    num_epochs=5,
    patience=5,
    is_binary=False
)


C:\Users\Vivian\AppData\Local\Temp\ipykernel_29020\1036823790.py:38: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()



=== Fold 1/5 ===


Fold 1 - Epoch 1:   0%|          | 0/771 [00:00<?, ?it/s]C:\Users\Vivian\AppData\Local\Temp\ipykernel_29020\1036823790.py:53: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Fold 1 - Epoch 1: 100%|██████████| 771/771 [03:32<00:00,  3.63it/s]


Epoch 1: Train Loss = 469.6805, Train Acc = 0.7911
✅ Model saved at epoch 1 with train acc: 0.7911


Fold 1 - Epoch 2: 100%|██████████| 771/771 [03:31<00:00,  3.64it/s]


...

Fold 5 - Epoch 5: 100%|██████████| 792/792 [03:34<00:00,  3.70it/s]
C:\Users\Vivian\AppData\Local\Temp\ipykernel_29020\1036823790.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental f

Epoch 5: Train Loss = 60.7324, Train Acc = 0.9670
No improvement for 2 epoch(s)


Fold 5 [Test]: 100%|██████████| 198/198 [00:42<00:00,  4.70it/s]

✅ All folds complete. Results saved.

📊 Cross-Validation Averages ± Std:
Best Train Accuracy: 0.9804 ± 0.0081
Test Accuracy: 0.7623 ± 0.1916
Test Precision: 0.7736 ± 0.2145
Test Recall: 0.7623 ± 0.1916
Test F1: 0.7468 ± 0.2240

📊 Per-Class F1 Scores:
Class 0: 0.4039 ± 0.4235
Class 1: 0.6606 ± 0.2303
Class 2: 0.2205 ± 0.3408
Class 3: 0.7522 ± 0.1716
Class 4: 0.8443 ± 0.1180
Class 5: 0.5160 ± 0.4326
Class 6: 0.4224 ± 0.3769
Class 7: 0.5010 ± 0.4112
📁 Appended summary rows to: C:\Users\Vivian\Documents\CONCH\breakhis_scripts\UNI_multiclass\UNI_fullft_results.csv



c:\Users\Vivian\anaconda3\envs\conch\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Vivian\anaconda3\envs\conch\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Vivian\anaconda3\envs\conch\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Vivian\anaconda3\e